Run MLP (first layer weights fixed) on mnist and compute bias and variance

In [5]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import queue
import os
import sys
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import copy 
import pandas as pd
class MLP(nn.Module):
    def __init__(self, p, d, o):
        """RF_models
        
        Args:
            p (int): the hidden size
            d (int): the input feature dimension
            o (int): the output dimension
        """
        super(MLP, self).__init__()

        self.fc1 = nn.Linear(d, p, bias=False)
        self.fc2 = nn.Linear(p, o, bias=False)
        self.p = p
        self.d = d
        self.o = o 
    def forward(self, x):
        o = F.relu(self.fc1(x))
        o = self.fc2(o)
        return o
class Ensemble_Two_Layer_NN(object):
    def __init__(self, n_classifiers, p, d=784, o=10):
        """Ensemble_Two_Layer_NN
        
        Args:
            p (int): the hidden size
            d (int, optional): the input feature dimension
            o (int, optional): the output dimension
            coef (float, optional): the ridge regression penalty coefficient
        """
        self.n_classifiers = n_classifiers
        self.p = p
        self.d = d 
        self.o = o 
        self.coef = coef
        self.learners = queue.LifoQueue(maxsize = self.n_classifiers)
        self.MODEL_TYPE = MLP
    def __len__(self):
        return len(self.learners.queue)
    
    def put_model_rho(self, model, rho):
        self.learners.put([model, rho])
    def get_init_model(self, cuda=True):
        model = self.MODEL_TYPE(self.p, self.d, self.o)
        if cuda:
            model.cuda()
        return model
    def cuda(self):
        if len(self) == 0:
            return 
        else:
            for model, rho in self.learners.queue:
                model.cuda()
            return
    def train(self):
        if len(self)!=0:
            for model, rho in self.learners.queue:
                model.train()
    def eval(self):
        if len(self)!=0:
            for model, rho in self.learners.queue:
                model.eval()
    def forward(self, x):
        Bs = x.size(0)
        if len(self) == 0:
            zeros = torch.zeros(Bs, self.o)
            zeros = zeros.to(x.device)
            return zeros
        else:
            outputs = torch.zeros(Bs, self.o)
            outputs = outputs.to(x.device) 
            for model, rho in self.learners.queue:
                output = model(x)
                outputs += rho*output
            return outputs


In [6]:
def get_subsample_dataset(trainset, subset):
    trainsubset = copy.deepcopy(trainset)
    trainsubset.data = [trainsubset.data[index] for index in subset]
    trainsubset.targets = [trainsubset.targets[index] for index in subset]
    return trainsubset
def fix_width_number(width, n_classifiers):
    return max(1, width//n_classifiers)

# Training
def train(net, trainset, permute_index, train_size, num_iters, lr, batch_size, coef):
    net.train()
    subsample_indexes = np.random.choice(permute_index, size=train_size)
    trainsubset = get_subsample_dataset(trainset, subsample_indexes)
    trainloader = torch.utils.data.DataLoader(trainsubset, batch_size=batch_size, shuffle=True)

    for i_c in range(net.n_classifiers):
        i_iter = 0
        model = net.get_init_model(cuda=True)
        rho = 1/net.n_classifiers
        optimizer = torch.optim.SGD(model.fc2.parameters(), lr=lr)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = num_iters//3, gamma = 0.1)
        while i_iter < num_iters:
            for inputs, targets in trainloader:
                Bs = inputs.size(0)
                inputs = inputs.reshape(Bs, -1)
                inputs, targets = inputs.cuda(), targets.cuda()
                targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
                targets_onehot.zero_()
                targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets_onehot)
                l2_reg = 0
                for param in model.fc2.parameters():
                    l2_reg += coef * torch.norm(param)
                mse_loss = loss.item()
                l2_loss = l2_reg.item()
                loss += l2_reg
                loss.backward()
                optimizer.step()
                lr_scheduler.step()
                string = "Train {} model: Iters [{}/{}] mse: {:.4f}, l2_loss: {:.4f}, train_loss:{:.4f}".format(i_c+1, i_iter, num_iters,  mse_loss, l2_loss, loss.item())
                sys.stdout.write(string+"\r")
                sys.stdout.flush()
                i_iter +=1
        net.put_model_rho(model, rho)
    # after all models were trained, estimate the mse error
    train_loss = 0
    correct = 0
    total = 0
    for inputs, targets in trainloader:
        Bs = inputs.size(0)
        inputs = inputs.reshape(Bs, -1)
        inputs, targets = inputs.cuda(), targets.cuda()
        targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
        targets_onehot.zero_()
        targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
        outputs = net.forward(inputs)
        loss = criterion(outputs, targets_onehot)
        train_loss = loss.item() * outputs.numel()
        _, predicted = outputs.max(1)
        correct = predicted.eq(targets).sum().item()
        total = targets.size(0)
    return train_loss/ total , 100. * correct / total

# Test
def test(net, testloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            Bs = inputs.size(0)
            inputs = inputs.reshape(Bs, -1)
            targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
            targets_onehot.zero_()
            targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
            outputs = net.forward(inputs)
            loss = criterion(outputs, targets_onehot)
            test_loss += loss.item() * outputs.numel()
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
    return test_loss / total, 100. * correct / total

def compute_bias_variance(net, testloader, trial, OUTPUST_SUM, OUTPUTS_SUMNORMSQUARED):
    net.eval()
    bias2 = 0
    variance = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            Bs = inputs.size(0)
            inputs = inputs.reshape(Bs, -1)
            targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
            targets_onehot.zero_()
            targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
            outputs = net.forward(inputs)
            OUTPUST_SUM[total:(total + targets.size(0)), :] += outputs
            OUTPUTS_SUMNORMSQUARED[total:total + targets.size(0)] += outputs.norm(dim=1) ** 2.0

            bias2 += (OUTPUST_SUM[total:total + targets.size(0), :] / (trial + 1) - targets_onehot).norm() ** 2.0
            variance += OUTPUTS_SUMNORMSQUARED[total:total + targets.size(0)].sum()/(trial + 1) - (OUTPUST_SUM[total:total + targets.size(0), :]/(trial + 1)).norm() ** 2.0
            total += targets.size(0)

    return bias2 / total, variance / total


In [7]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)
# loss definition
criterion = nn.MSELoss(reduction='mean').cuda()

def run_exps_sgd(train_sizes, N_Ds, P_Ns, trainset, test_size, feature_dim, num_classes, num_trials, coef,
             outdir, save_csv, num_iters, lr, batch_size_list, K = 1):
    df = pd.DataFrame()
    for batch_size in batch_size_list:
        for train_size in train_sizes:
            hidden_sizes = P_Ns * train_size
            hidden_sizes = np.unique([int(np.around(x)) for x in hidden_sizes])
            for hidden_size in hidden_sizes:
                TRAIN_ACC_SUM = 0.0
                TEST_ACC_SUM = 0.0
                TRAIN_LOSS_SUM = 0.0
                TEST_LOSS_SUM = 0.0
                permute_index = np.random.permutation(len(trainset))
                OUTPUST_SUM = torch.Tensor(test_size, num_classes).zero_().cuda()
                OUTPUTS_SUMNORMSQUARED = torch.Tensor(test_size).zero_().cuda()
                for trial in range(num_trials):
                    net = Ensemble_Two_Layer_NN(n_classifiers = K, p = fix_width_number(hidden_size, K), d=feature_dim, o=num_classes)
                    net.cuda()
                    train_loss, train_acc = train(net, trainset, permute_index, train_size,
                                                 num_iters, lr, batch_size, coef)
                    test_loss, test_acc = test(net, testloader)

                    TRAIN_LOSS_SUM += train_loss
                    TEST_LOSS_SUM += test_loss
                    TRAIN_ACC_SUM += train_acc
                    TEST_ACC_SUM += test_acc

                    # compute bias and variance
                    bias2, variance = compute_bias_variance(net, testloader, trial, OUTPUST_SUM, OUTPUTS_SUMNORMSQUARED)
                    variance_unbias = variance * num_trials / (num_trials - 1.0)
                    bias2_unbias = TEST_LOSS_SUM / (trial + 1) - variance_unbias
                    print('Train size: [{}] hidden size: [{}] batch size: [{}] trial: {}, train_loss: {:.6f}, train acc: {}, test loss: {:.6f}, test acc: {}, bias2: {}, variance: {}'.format(
                        train_size, hidden_size, batch_size,
                        trial, TRAIN_LOSS_SUM / (trial + 1), TRAIN_ACC_SUM / (trial + 1), TEST_LOSS_SUM / (trial + 1),
                        TEST_ACC_SUM / (trial + 1), bias2_unbias, variance_unbias))
                    torch.cuda.empty_cache()
                print('#'*50)
                df = df.append({'train_size': train_size, 'hidden_size':hidden_size, 'batch_size': batch_size,
                                'train_loss': TRAIN_LOSS_SUM / (trial + 1), 'train_acc': TRAIN_ACC_SUM / (trial + 1),
                                'test_loss': TEST_LOSS_SUM / (trial + 1), 'test_acc': TEST_ACC_SUM / (trial + 1), 
                               'variance': variance_unbias.item(),
                               'bias2': bias2_unbias.item()}, ignore_index=True)
                df.to_csv(os.path.join(outdir, save_csv))
    df.to_csv(os.path.join(outdir, save_csv))

In [8]:
num_classes = 10
num_trials = 50
coef = 0.0001
N_Ds = [1]
feature_dim = 784
lr = 0.01 
batch_size_list = [1, 784]
train_sizes = [int(np.around(x*feature_dim)) for x in N_Ds]
test_size = 10000
P_Ns = 10** np.linspace(-2, 1, 50)

for num_iters in [500, 5000]:
    outdir = 'mnist_SGD/num_iters_{}_coef={}'.format(num_iters, coef)
    print(outdir)
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    run_exps_sgd(train_sizes, N_Ds, P_Ns, trainset, test_size, feature_dim, num_classes, num_trials, coef,
             outdir, 'singleNN_output.csv', num_iters = num_iters, lr=lr, batch_size_list=batch_size_list,  K = 1)
    run_exps_sgd(train_sizes, N_Ds, P_Ns, trainset, test_size, feature_dim, num_classes, num_trials, coef,
                 outdir, 'ensembleNNK=2_output.csv', num_iters = num_iters, lr=lr, batch_size_list=batch_size_list,  K = 2)

mnist_SGD/num_iters_500_coef=0.0001
Train size: [784] hidden size: [8] batch size: [10] trial: 0, train_loss: 1.136294, train acc: 50.0, test loss: 1.374105, test acc: 9.57, bias2: 1.3741050958633423, variance: 5.838822203507732e-10
Train size: [784] hidden size: [8] batch size: [10] trial: 1, train_loss: 1.194225, train acc: 25.0, test loss: 1.341597, test acc: 11.01, bias2: 1.1910598278045654, variance: 0.15053719282150269
Train size: [784] hidden size: [8] batch size: [10] trial: 2, train_loss: 1.210376, train acc: 25.0, test loss: 1.362627, test acc: 11.99, bias2: 1.1203193664550781, variance: 0.24230729043483734
Train size: [784] hidden size: [8] batch size: [10] trial: 3, train_loss: 1.194897, train acc: 18.75, test loss: 1.351751, test acc: 11.334999999999999, bias2: 1.0921508073806763, variance: 0.259600430727005
Train size: [784] hidden size: [8] batch size: [10] trial: 4, train_loss: 1.184570, train acc: 25.0, test loss: 1.340106, test acc: 11.657999999999998, bias2: 1.054060

KeyboardInterrupt: 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
font = {
        'size'   : 18}
matplotlib.rc('font', **font)
figsize = (16, 5)
import seaborn as sns
sns.set_style('darkgrid')
import pandas as pd

def plot_bias_var(df, N_D, ymin=0, ymax=1.0):
    fig1, axes1 = plt.subplots(1, 3, figsize=figsize)
    axes1[0].set_xscale('log')
    axes1[1].set_xscale('log')
    axes1[2].set_xscale('log')
    cur_df = df[df['train_size']/feature_dim==N_D]
    test_loss = cur_df['test_loss']
    bias2 = cur_df['bias2']
    var = cur_df['variance']
    P_N = cur_df['hidden_size']/cur_df['train_size']
    axes1[0].plot(P_N, test_loss)
    axes1[0].set_xlabel("P/N")
    axes1[0].set_ylabel("Test Loss")
    axes1[0].set_ylim(ymin, ymax)
    axes1[1].plot(P_N, bias2)
    axes1[1].set_xlabel("P/N")
    axes1[1].set_ylabel("Bias Square")
    axes1[1].set_ylim(ymin, ymax)
    axes1[2].plot(P_N, var)
    axes1[2].set_xlabel("P/N")
    axes1[2].set_ylabel("Variance")
    axes1[2].set_ylim(ymin, ymax)
    fig1.suptitle("Bias-Variance Decomposition (N/D={:.2f})".format(N_D))
    plt.show()
def plot_single_vs_ensemble(dfs_list, Ks_list, N_D, feature_dim, ymin=0, ymax=1.0):
    assert len(dfs_list) == len(Ks_list)
    fig1, axes1 = plt.subplots(1, 3, figsize=figsize)
    for i in range(3):
        axes1[i].set_xscale('log')
    dfs_list = [df[df['train_size']/feature_dim==N_D] for df in dfs_list]
    for cur_df, K in zip(dfs_list, Ks_list):
        test_loss = cur_df['test_loss']
        bias2 = cur_df['bias2']
        var = cur_df['variance']
        P_N = cur_df['hidden_size']/cur_df['train_size']
        axes1[0].plot(P_N, test_loss, label='K={}'.format(K))
        axes1[1].plot(P_N, bias2, label='K={}'.format(K))
        axes1[2].plot(P_N, var, label='K={}'.format(K))
    
    axes1[0].set_xlabel("P/N")
    axes1[0].set_ylabel("Test Loss")
    axes1[0].set_ylim(ymin, ymax)
    
    axes1[1].set_xlabel("P/N")
    axes1[1].set_ylabel("Bias Square")
    axes1[1].set_ylim(ymin, ymax)
    
    axes1[2].set_xlabel("P/N")
    axes1[2].set_ylabel("Variance")
    axes1[2].set_ylim(ymin, ymax)
    fig1.suptitle("Bias-Variance Decomposition (N/D={:.2f})".format(N_D))
    plt.legend()
    plt.show()


In [ ]:
# K2_df = pd.read_csv(os.path.join(outdir, 'ensembleNNK=2_output.csv'))
# K1_df = pd.read_csv(os.path.join(outdir, 'singleNN_output.csv'))
# plot_single_vs_ensemble([K1_df, K2_df], [1, 2], N_Ds[0], 784,)